In [1]:
from AutoML import AutoML, DataNet, AutoMLDataset
#DataNet().restart_datanodes()
dataset = AutoMLDataset(name="person_classification", semantics=[113988, 113989, 113990], crops=True) #, override=True)

/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/reminiz/ReminizML2/python_env/lib/pyt

In [2]:
import ray
ray.init(address="auto", ignore_reinit_error=True)

2021-01-03 13:10:25,761	INFO worker.py:634 -- Connecting to existing Ray cluster at address: 192.168.8.51:6379
2021-01-03 13:10:25,762	ERROR worker.py:643 -- Calling ray.init() again after it has already been called.


In [3]:
add_images_stage = 5000

In [4]:
# get the indexes to emulate the labeled set

import numpy as np
import random

list_classes = list(dataset.dataset_header['train_images'].keys())
nb_classes = len(list_classes)        
classes={}
files = []
labels = []
nb_img_per_class = np.zeros(nb_classes, dtype = np.int32)
classes_cardinality = {}

for class_name in list_classes:
    class_index = list_classes.index(class_name)
    classes_cardinality[class_name] = len(dataset.dataset_header['train_images'][class_name])
    nb_img_per_class[class_index] = classes_cardinality[class_name]
    start = len(files)
    id_files = dataset.dataset_header["train_images"][class_name]
    files+=id_files
    labels+=[class_index] * len(id_files)
    end = len(files)
    classes[class_index]=range(start, end)

    
all_files = np.array(files)
all_labels = np.array(labels)
indices = list(range(len(files)))
random.seed(101)
random.shuffle(indices)
labeled_set = indices[:add_images_stage ]
unlabeled_set = indices[add_images_stage :]


In [5]:
# load defaulft config
import yaml

config_path = '/mnt/Ressources/Andres/Temp_active/configs/Datanet_random.yml'

with open(config_path) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    config = yaml.load(file, Loader=yaml.FullLoader)

print(config)

{'DATASET': {'Data_augementation': True, 'original_size': 256, 'pad': False, 'sampling': 'log_proportional', 'random_crop': True, 'random_crop_margin': 0.1, 'random_flip': True, 'random_greyscale': False, 'random_hue': False, 'rot90': False, 'random_brightness': False, 'random_saturation': False}, 'NETWORK': {'Backbone': 'ResNet50', 'INPUT_SIZE': 256, 'MARGIN': 1.0, 'embedding_size': 128}, 'PROJECT': {'dataset_name': 'person_classification', 'group': 'AL_v2', 'group_dir': '/mnt/Ressources/Andres/runs/AL_v2', 'project': 'Active_Learning_Datanet', 'project_dir': '/mnt/Ressources/Andres/runs'}, 'TEST': {'batch_size': 16}, 'TRAIN': {'EPOCH_SLIT': 80, 'EPOCH_WARMUP': 2, 'EPOCH_WHOLE': 120, 'MILESTONES': [140, 180], 'batch_size': 16, 'lr': 0.0025, 'test_each': 1, 'transfer_weight_path': '/mnt/Ressources/Andres/runs/imagenet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', 'weight_lossnet_loss': 1.0, 'weight_decay': 2e-05, 'apply_weight_decay': False}}


In [6]:
import time
import os

In [7]:
from train_agent_classification_AL import Active_Learning_train


In [ ]:
for num_run in range(10):

    prev_num_run = num_run-1
    if num_run==0:
        resume_model_path = False
    else:
        resume_model_path = os.path.join(config['PROJECT']['group_dir'],'Stage_'+str(prev_num_run),'checkpoints','checkpoint.200.hdf5')
        
    stage_files = list(all_files[labeled_set])
    stage_labels = list(all_labels[labeled_set])
        
    Active_Learning = Active_Learning_train.remote(config, 
                                            stage_files,
                                            stage_labels,
                                            num_run=num_run,
                                            resume_model_path=resume_model_path,
                                            resume = False)
    
    Active_Learning.start_training.remote()
    
    # Wait util the model is training
    while True:
        time.sleep(10)
        try:
            progress_id = Active_Learning.isTraining.remote()
            response = ray.get(progress_id)
            break
        except:
            pass
        
    # wait until the model finish training
    while True:
        time.sleep(10)
        progress_id = Active_Learning.isTraining.remote()
        response = ray.get(progress_id)
        if not response:
            break
    
    Active_Learning.__ray_terminate__.remote()
    
    del Active_Learning

    labeled_set += unlabeled_set[: add_images_stage]
    unlabeled_set = unlabeled_set[add_images_stage :]


(pid=20336, ip=192.168.8.52) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=20336, ip=192.168.8.52)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=20336, ip=192.168.8.52) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=20336, ip=192.168.8.52)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=20336, ip=192.168.8.52) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, i

(pid=19165, ip=192.168.8.54)  - 84s - loss: 0.3487 - c_pred_loss: 0.2360 - l_pred_w_loss: 0.1128 - l_pred_s_loss: 0.1128 - c_pred_sparse_categorical_accuracy: 0.9101 - l_pred_w_MAE_Lossnet: 4.3904 - l_pred_s_MAE_Lossnet: 4.3904
(pid=19165, ip=192.168.8.54) Epoch 9/200


(pid=20336, ip=192.168.8.52) 2021-01-03 13:10:51.435810: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=20336, ip=192.168.8.52) 2021-01-03 13:10:51.465599: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3300305000 Hz
(pid=20336, ip=192.168.8.52) 2021-01-03 13:10:51.466535: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x51441a0 executing computations on platform Host. Devices:
(pid=20336, ip=192.168.8.52) 2021-01-03 13:10:51.466591: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
(pid=20336, ip=192.168.8.52) 2021-01-03 13:11:15.669781: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x5185fd0 executing computations on platform CUDA. Devices:
(pid=20336, ip=192.168.8.52) 2021-01-03 13:11:15.669833: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0)

(pid=20336, ip=192.168.8.52) Subset set to : train
(pid=20336, ip=192.168.8.52) Train_Stage_0 The backbone is:  ResNet50
(pid=20336, ip=192.168.8.52) Train_Stage_0 (transfer learning) Loading weigths by name from:  /mnt/Ressources/Andres/runs/imagenet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5


(pid=20336, ip=192.168.8.52) 2021-01-03 13:11:24,623	INFO worker.py:634 -- Connecting to existing Ray cluster at address: 192.168.8.51:6379
(pid=20336, ip=192.168.8.52) 2021-01-03 13:11:24,623	ERROR worker.py:643 -- Calling ray.init() again after it has already been called.
(pid=20336, ip=192.168.8.52) WARNING:tensorflow:Output "Embedding" missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to "Embedding".
(pid=20336, ip=192.168.8.52) WARNING:tensorflow:Output "Embedding" missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to "Embedding".


...

(pid=24182, ip=192.168.8.55) wandb: Tracking run with wandb version 0.8.36
(pid=24182, ip=192.168.8.55) wandb: Wandb version 0.10.12 is available!  To upgrade, please run:
(pid=24182, ip=192.168.8.55) wandb:  $ pip install wandb --upgrade
(pid=24182, ip=192.168.8.55) wandb: Run data is saved locally in ../../../Trainings/wandb/wandb/run-20210103_220813-3d9ir3ww
(pid=24182, ip=192.168.8.55) wandb: Syncing run 3d9ir3ww
(pid=24182, ip=192.168.8.55) wandb: ⭐️ View project at http://192.168.8.51:8080/reminiz/Face%20Detection
(pid=24182, ip=192.168.8.55) wandb: 🚀 View run at http://192.168.8.51:8080/reminiz/Face%20Detection/runs/3d9ir3ww
(pid=24182, ip=192.168.8.55) wandb: Run `wandb off` to turn off syncing.
(pid=24182, ip=192.168.8.55) 
(pid=24182, ip=192.168.8.55) WARNING:tensorflow:From /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in

(pid=24182, ip=192.168.8.55) {0: 'Face'}


(pid=24182, ip=192.168.8.55) 2021-01-03 23:08:18.421705: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=24182, ip=192.168.8.55) 2021-01-03 23:08:18.451599: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3600075000 Hz
(pid=24182, ip=192.168.8.55) 2021-01-03 23:08:18.452349: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x72addf0 executing computations on platform Host. Devices:
(pid=24182, ip=192.168.8.55) 2021-01-03 23:08:18.452375: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
(pid=24182, ip=192.168.8.55) 2021-01-03 23:08:18.580313: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x72ea0b0 executing computations on platform CUDA. Devices:
(pid=24182, ip=192.168.8.55) 2021-01-03 23:08:18.580357: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0)

(pid=24182, ip=192.168.8.55) Restoring from /mnt/Trainings/models/face_detection_face_1/checkpoint/epoch14.ckpt-14


  0% 0/100 [00:00<?, ?it/s]E0103 23:08:32.502254 24182 24194 task_manager.cc:323] Task failed: IOError: cancelling all pending tasks of dead actor: Type=ACTOR_TASK, Language=PYTHON, Resources: {}, function_descriptor={type=PythonFunctionDescriptor, module_name=AutoML.datanode, class_name=DataNode, function_name=get_image, function_hash=}, task_id=a09ad39b05626e4b4d7dd2d001000000, task_name=DataNode.get_image(), job_id=01000000, num_args=6, num_returns=2, actor_task_spec={actor_id=4d7dd2d001000000, actor_caller_id=ffffffffffffffff73b311d76f000000, actor_counter=0}
  1% 1/100 [00:04<08:05,  4.90s/it]5) 
  3% 3/100 [00:05<05:35,  3.46s/it]5) 
  4% 4/100 [00:05<04:00,  2.50s/it]5) 
  5% 5/100 [00:05<02:52,  1.82s/it]5) 
  7% 7/100 [00:05<02:00,  1.29s/it]5) 
  9% 9/100 [00:06<01:26,  1.05it/s]5) 
 10% 10/100 [00:06<01:03,  1.41it/s]) 
 11% 11/100 [00:06<00:48,  1.85it/s]) 
 12% 12/100 [00:06<00:37,  2.36it/s]) 
 13% 13/100 [00:06<00:29,  2.92it/s]) 
 14% 14/100 [00:06<00:24,  3.46it/s]) 
 